## 1. Environment Setup

In [13]:
# Environment setup
from pathlib import Path
from typing import Optional
import pandas as pd
import numpy as np

try:
    from dotenv import load_dotenv
    DOTENV_AVAILABLE = True
except Exception:
    DOTENV_AVAILABLE = False

# Helper to find project root
def _find_root(start: Optional[Path] = None) -> Path:
    p = start or Path.cwd()
    for _ in range(6):
        if (p / 'data').exists() or (p / '.git').exists() or (p / 'notebooks').exists():
            return p
        p = p.parent
    return Path.cwd()

# Resolve project directories consistently
ROOT = _find_root()
DATA_DIR = ROOT / 'data' / 'raw'
INTERIM_DIR = ROOT / 'data' / 'interim'
PROCESSED_DIR = ROOT / 'data' / 'processed'
FIG_DIR = ROOT / 'reports' / 'figures'
for d in [DATA_DIR, INTERIM_DIR, PROCESSED_DIR, FIG_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"\n🎯 Environment setup complete")
print(f"   ROOT: {ROOT}")
print(f"   DATA_DIR: {DATA_DIR}")
print(f"   INTERIM_DIR: {INTERIM_DIR}")


🎯 Environment setup complete
   ROOT: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks
   DATA_DIR: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks\data\raw
   INTERIM_DIR: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks\data\interim


## 2. Load Team Name Normalization

We need the team name mapping to ensure consistent team names across all seasons.
These functions were defined in the regular season collection notebook.

In [14]:
# Team Name Mapping - same as in regular season notebook
TEAM_NAME_MAP = {
    # Abbreviations to full names
    'ASH': 'F.C. Ashdod',
    'BEI': 'Beitar Jerusalem',
    'BnS': 'Bnei Sakhnin',
    'BnY': 'Bnei Yehuda',
    'HAS': 'Hapoel Ashkelon',
    'HBS': "Hapoel Be'er Sheva",
    'HHA': 'Hapoel Haifa',
    'HKS': 'Hapoel Kfar Saba',
    'HRA': "Hapoel Ra'anana",
    'HTA': 'Hapoel Tel Aviv',
    'IKS': 'Ironi Kiryat Shmona',
    'MHA': 'Maccabi Haifa',
    'MPT': 'Maccabi Petah Tikva',
    'MTA': 'Maccabi Tel Aviv',
    'HPT': 'Hapoel Petah Tikva',
    'HRG': 'Hapoel Ramat Gan',
    'HRH': 'Hapoel Ramat HaSharon',
    'HRL': 'Rishon LeZion',
    'MAN': 'Maccabi Ahi Nazareth',
    'MBR': 'Maccabi Bnei Reineh',
    'SNZ': 'Sektzia Ness Ziona',
    'HAK': 'Hapoel Acre',
    'MHE': 'Maccabi Herzliya',
    'MNE': 'Maccabi Netanya',
    'HAR': 'Hapoel Raanana',
    'HAC': 'Hapoel Acre',
    'IRH': 'Ironi Ramat HaSharon',
    'HAH': 'Hapoel Hadera',
    'NES': 'Ness Ziona',
    'HJE': 'Hapoel Jerusalem',
    'HNG': 'Hapoel Nof HaGalil',
    'ITI': 'Ironi Tiberias',
    
    # Name variants to canonical names
    'Ashdod': 'F.C. Ashdod',
    'F.C. Ironi Ashdod': 'F.C. Ashdod',
    'Ness Ziona': 'Sektzia Ness Ziona',
    'Ironi Nir Ramat HaSharon': 'Ironi Ramat HaSharon',
    'Hakoah Amidar Ramat Gan': 'Hapoel Ramat Gan',
    'Hapoel Rishon LeZion': 'Rishon LeZion',
    'Hapoel Raanana': "Hapoel Ra'anana",
    
    # Full names map to themselves
    'F.C. Ashdod': 'F.C. Ashdod',
    'Beitar Jerusalem': 'Beitar Jerusalem',
    'Bnei Sakhnin': 'Bnei Sakhnin',
    'Bnei Yehuda': 'Bnei Yehuda',
    'Hapoel Ashkelon': 'Hapoel Ashkelon',
    "Hapoel Be'er Sheva": "Hapoel Be'er Sheva",
    'Hapoel Haifa': 'Hapoel Haifa',
    'Hapoel Kfar Saba': 'Hapoel Kfar Saba',
    "Hapoel Ra'anana": "Hapoel Ra'anana",
    'Hapoel Tel Aviv': 'Hapoel Tel Aviv',
    'Ironi Kiryat Shmona': 'Ironi Kiryat Shmona',
    'Maccabi Haifa': 'Maccabi Haifa',
    'Maccabi Petah Tikva': 'Maccabi Petah Tikva',
    'Maccabi Tel Aviv': 'Maccabi Tel Aviv',
    'Hapoel Petah Tikva': 'Hapoel Petah Tikva',
    'Hapoel Ramat Gan': 'Hapoel Ramat Gan',
    'Hapoel Ramat HaSharon': 'Hapoel Ramat HaSharon',
    'Rishon LeZion': 'Rishon LeZion',
    'Maccabi Ahi Nazareth': 'Maccabi Ahi Nazareth',
    'Maccabi Bnei Reineh': 'Maccabi Bnei Reineh',
    'Sektzia Ness Ziona': 'Sektzia Ness Ziona',
    'Hapoel Acre': 'Hapoel Acre',
    'Maccabi Herzliya': 'Maccabi Herzliya',
    'Maccabi Netanya': 'Maccabi Netanya',
    'Ironi Ramat HaSharon': 'Ironi Ramat HaSharon',
    'Hapoel Hadera': 'Hapoel Hadera',
    'Hapoel Jerusalem': 'Hapoel Jerusalem',
    'Hapoel Nof HaGalil': 'Hapoel Nof HaGalil',
    'Ironi Tiberias': 'Ironi Tiberias',
}

def normalize_team_names(df, name_map=TEAM_NAME_MAP):
    """Normalize team names by converting abbreviations and variants to full names."""
    df = df.copy()
    df['home_team'] = df['home_team'].map(lambda x: name_map.get(x, x))
    df['away_team'] = df['away_team'].map(lambda x: name_map.get(x, x))
    return df

def apply_season_specific_fixes(df, season):
    """Apply season-specific Wikipedia data corrections."""
    df = df.copy()
    if season == '2006/07':
        # Fix: Hapoel Ramat Gan should be Hapoel Acre in 2006/07
        df.loc[df['home_team'] == 'Hapoel Ramat Gan', 'home_team'] = 'Hapoel Acre'
        df.loc[df['away_team'] == 'Hapoel Ramat Gan', 'away_team'] = 'Hapoel Acre'
    elif season == '2008/09':
        # Fix: Hapoel Ramat Gan should be Hapoel Ra'anana in 2008/09
        df.loc[df['home_team'] == 'Hapoel Ramat Gan', 'home_team'] = "Hapoel Ra'anana"
        df.loc[df['away_team'] == 'Hapoel Ramat Gan', 'away_team'] = "Hapoel Ra'anana"
    return df

print("✅ Team name normalization functions loaded")

✅ Team name normalization functions loaded


## 3. Load Match Data

Load the combined Wikipedia match data that was collected in the regular season notebook.

In [15]:
# Load all match data from Wikipedia
matches_path = DATA_DIR / "matches_all_seasons_ligat_haal_wikipedia.csv"

if not matches_path.exists():
    print(f"❌ Match data not found: {matches_path}")
    print(f"\n⚠️  Please run the '01_regular_season_collection.ipynb' notebook first to collect match data.")
    raise FileNotFoundError(f"Required file not found: {matches_path}")

# Load and normalize
all_matches = pd.read_csv(matches_path)
print(f"✅ Loaded {len(all_matches)} matches from {all_matches['season'].nunique()} seasons")
print(f"   Season range: {all_matches['season'].min()} to {all_matches['season'].max()}")

# Normalize team names (convert abbreviations to full names)
all_matches = normalize_team_names(all_matches, TEAM_NAME_MAP)

# Apply season-specific fixes
for season_name in all_matches['season'].unique():
    season_mask = all_matches['season'] == season_name
    season_data = all_matches[season_mask].copy()
    all_matches.loc[season_mask] = apply_season_specific_fixes(season_data, season_name)

print(f"✅ Team names normalized and season-specific fixes applied")
print(f"\n📊 Data sample:")
display(all_matches.head())

✅ Loaded 3533 matches from 20 seasons
   Season range: 2006/07 to 2025/26
✅ Team names normalized and season-specific fixes applied

📊 Data sample:


,season,season_year,home_team,away_team,home_goals,away_goals,goal_diff,result,home_points,away_points
0,2006/07,2006,Beitar Jerusalem,Bnei Yehuda,0,0,0,D,1,1
1,2006/07,2006,Beitar Jerusalem,F.C. Ashdod,2,0,2,H,3,0
2,2006/07,2006,Beitar Jerusalem,Hapoel Acre,0,0,0,D,1,1
3,2006/07,2006,Beitar Jerusalem,Hapoel Kfar Saba,2,0,2,H,3,0
4,2006/07,2006,Beitar Jerusalem,Hapoel Petah Tikva,2,0,2,H,3,0


## 4. Title Race Analysis Function

Calculate league standings after each round and track leadership changes.
This is the core function for analyzing the "title race" throughout a season.

In [16]:
def calculate_league_table_by_round(matches_df, season_str="2016/17"):
    """
    Calculate league standings after each round/matchday and track leadership changes.
    
    Args:
        matches_df: DataFrame with match results (must have normalized team names)
        season_str: Season to analyze (e.g., "2016/17")
    
    Returns:
        - standings_by_round: dict mapping round_num -> DataFrame of standings
        - leadership_changes: list of tuples (round_num, new_leader)
    
    Note: Team names should already be normalized (full names, not abbreviations).
    """
    # Filter for the specific season
    season_matches = matches_df[matches_df['season'] == season_str].copy()
    
    if len(season_matches) == 0:
        print(f"❌ No matches found for season {season_str}")
        return None, None
    
    # Get all unique teams from both home and away columns
    home_teams = set(season_matches['home_team'].unique())
    away_teams = set(season_matches['away_team'].unique())
    teams = sorted(home_teams | away_teams)  # Union of both sets
    n_teams = len(teams)
    
    print(f"📊 Processing {season_str}: {len(season_matches)} matches, {n_teams} teams")
    
    # In Ligat Ha'al, typically 14 teams play 26 rounds in regular season
    # Each round has n_teams/2 matches (e.g., 14 teams = 7 matches per round)
    # Since we don't have actual dates, we estimate round numbers
    season_matches = season_matches.reset_index(drop=True)
    matches_per_round = n_teams // 2 if n_teams % 2 == 0 else (n_teams + 1) // 2
    
    # Assign estimated round numbers based on position in dataset
    season_matches['round_num'] = (season_matches.index // matches_per_round) + 1
    max_round = season_matches['round_num'].max()
    
    # Initialize standings tracker
    standings_by_round = {}
    current_leader = None
    leadership_changes = []
    
    # Calculate standings after each round
    for round_num in sorted(season_matches['round_num'].unique()):
        # Get all matches up to and including this round
        matches_so_far = season_matches[season_matches['round_num'] <= round_num]
        
        # Initialize team stats
        stats = {team: {
            'played': 0, 'won': 0, 'drawn': 0, 'lost': 0, 
            'gf': 0, 'ga': 0, 'gd': 0, 'points': 0
        } for team in teams}
        
        # Calculate stats from matches
        for _, match in matches_so_far.iterrows():
            home = match['home_team']
            away = match['away_team']
            home_goals = match['home_goals']
            away_goals = match['away_goals']
            
            # Update home team
            stats[home]['played'] += 1
            stats[home]['gf'] += home_goals
            stats[home]['ga'] += away_goals
            stats[home]['gd'] = stats[home]['gf'] - stats[home]['ga']
            
            # Update away team
            stats[away]['played'] += 1
            stats[away]['gf'] += away_goals
            stats[away]['ga'] += home_goals
            stats[away]['gd'] = stats[away]['gf'] - stats[away]['ga']
            
            # Update points
            if home_goals > away_goals:  # Home win
                stats[home]['won'] += 1
                stats[home]['points'] += 3
                stats[away]['lost'] += 1
            elif away_goals > home_goals:  # Away win
                stats[away]['won'] += 1
                stats[away]['points'] += 3
                stats[home]['lost'] += 1
            else:  # Draw
                stats[home]['drawn'] += 1
                stats[away]['drawn'] += 1
                stats[home]['points'] += 1
                stats[away]['points'] += 1
        
        # Convert to DataFrame and sort by points, then goal difference, then goals scored
        standings = pd.DataFrame.from_dict(stats, orient='index')
        standings.index.name = 'team'
        standings = standings.reset_index()
        standings = standings.sort_values(['points', 'gd', 'gf'], ascending=[False, False, False])
        standings['position'] = range(1, len(standings) + 1)
        
        standings_by_round[int(round_num)] = standings
        
        # Track leadership changes
        new_leader = standings.iloc[0]['team']
        if new_leader != current_leader:
            leadership_changes.append((int(round_num), new_leader))
            current_leader = new_leader
    
    return standings_by_round, leadership_changes

print("✅ Title race analysis function defined")

✅ Title race analysis function defined


## 5. Analyze Single Season

Analyze the title race for a specific season.

In [17]:
# Analyze specific season
season = "2016/17"
print(f"🏆 Analyzing Title Race: {season}")
print("="*80)

standings_by_round, leadership_changes = calculate_league_table_by_round(all_matches, season)

if standings_by_round and leadership_changes:
    print(f"\n📊 League Leadership Analysis - {season}")
    print("="*80)
    print(f"\n🔄 Leadership Changes: {len(leadership_changes) - 1}")
    print(f"   (First leader doesn't count as a 'change')\n")
    
    print("Round-by-round first place:")
    for round_num, leader in leadership_changes:
        print(f"  • Round {round_num:2d}: {leader}")
    
    # Show final standings
    final_round = max(standings_by_round.keys())
    print(f"\n📋 Final Standings After Round {final_round}:")
    final = standings_by_round[final_round]
    display(final[['position', 'team', 'played', 'won', 'drawn', 'lost', 'gf', 'ga', 'gd', 'points']].head(10))
    
    # Calculate statistics
    print(f"\n📈 Season Statistics:")
    print(f"  • Total rounds: {len(standings_by_round)}")
    print(f"  • Teams: {len(final)}")
    print(f"  • Leader: {final.iloc[0]['team']} ({final.iloc[0]['points']:.0f} pts, {final.iloc[0]['played']:.0f} games)")
    print(f"  • Runner-up: {final.iloc[1]['team']} ({final.iloc[1]['points']:.0f} pts, {final.iloc[1]['played']:.0f} games)")
    print(f"  • Points gap: {final.iloc[0]['points'] - final.iloc[1]['points']:.0f} pts")
    
    print(f"\n⚠️  Note: Round numbers are estimated (Wikipedia doesn't include dates)")
    print(f"   Actual match sequencing may differ slightly.")
else:
    print(f"❌ Failed to analyze season {season}")

🏆 Analyzing Title Race: 2016/17
📊 Processing 2016/17: 182 matches, 14 teams

📊 League Leadership Analysis - 2016/17

🔄 Leadership Changes: 3
   (First leader doesn't count as a 'change')

Round-by-round first place:
  • Round  1: F.C. Ashdod
  • Round  3: Beitar Jerusalem
  • Round  8: Bnei Sakhnin
  • Round 10: Hapoel Be'er Sheva

📋 Final Standings After Round 26:

📊 League Leadership Analysis - 2016/17

🔄 Leadership Changes: 3
   (First leader doesn't count as a 'change')

Round-by-round first place:
  • Round  1: F.C. Ashdod
  • Round  3: Beitar Jerusalem
  • Round  8: Bnei Sakhnin
  • Round 10: Hapoel Be'er Sheva

📋 Final Standings After Round 26:


,position,team,played,won,drawn,lost,gf,ga,gd,points
5,1,Hapoel Be'er Sheva,26,18,5,3,54,13,41,59
13,2,Maccabi Tel Aviv,26,17,5,4,45,19,26,56
12,3,Maccabi Petah Tikva,26,13,9,4,36,23,13,48
0,4,Beitar Jerusalem,26,10,10,6,34,27,7,40
1,5,Bnei Sakhnin,26,10,9,7,26,26,0,39
11,6,Maccabi Haifa,26,10,8,8,30,25,5,38
10,7,Ironi Kiryat Shmona,26,9,8,9,35,33,2,35
6,8,Hapoel Haifa,26,8,4,14,29,36,-7,28
3,9,F.C. Ashdod,26,6,10,10,15,26,-11,28
8,10,Hapoel Ra'anana,26,7,7,12,14,29,-15,28



📈 Season Statistics:
  • Total rounds: 26
  • Teams: 14
  • Leader: Hapoel Be'er Sheva (59 pts, 26 games)
  • Runner-up: Maccabi Tel Aviv (56 pts, 26 games)
  • Points gap: 3 pts

⚠️  Note: Round numbers are estimated (Wikipedia doesn't include dates)
   Actual match sequencing may differ slightly.


## 6. Multi-Season Title Race Analysis

Analyze leadership changes across all available seasons.

In [18]:
# Analyze all seasons
print("🏆 Multi-Season Title Race Analysis")
print("="*80)

all_seasons = sorted(all_matches['season'].unique())
season_summary = []

for season_str in all_seasons:
    standings_by_round, leadership_changes = calculate_league_table_by_round(all_matches, season_str)
    
    if standings_by_round and leadership_changes:
        final_round = max(standings_by_round.keys())
        final_standings = standings_by_round[final_round]
        champion = final_standings.iloc[0]
        runner_up = final_standings.iloc[1]
        
        season_summary.append({
            'season': season_str,
            'champion': champion['team'],
            'champion_points': int(champion['points']),
            'runner_up': runner_up['team'],
            'runner_up_points': int(runner_up['points']),
            'points_gap': int(champion['points'] - runner_up['points']),
            'leadership_changes': len(leadership_changes) - 1,  # First doesn't count as change
            'total_rounds': len(standings_by_round)
        })

print()  # Newline after processing messages

# Create summary DataFrame
summary_df = pd.DataFrame(season_summary)

print(f"\n📊 Title Race Summary ({len(summary_df)} seasons):")
print("="*80)
display(summary_df)

# Save to interim folder for later use
summary_path = INTERIM_DIR / "title_race_summary.csv"
summary_df.to_csv(summary_path, index=False, encoding='utf-8-sig')
print(f"\n✅ Summary saved to: {summary_path}")

# Some interesting statistics
print(f"\n📈 Interesting Statistics:")
print(f"  • Most competitive season: {summary_df.loc[summary_df['leadership_changes'].idxmax(), 'season']} ({summary_df['leadership_changes'].max()} leadership changes)")
print(f"  • Most dominant win: {summary_df.loc[summary_df['points_gap'].idxmax(), 'season']} ({summary_df['points_gap'].max()} points gap)")
print(f"  • Closest finish: {summary_df.loc[summary_df['points_gap'].idxmin(), 'season']} ({summary_df['points_gap'].min()} points gap)")
print(f"  • Average leadership changes per season: {summary_df['leadership_changes'].mean():.1f}")
print(f"  • Average points gap: {summary_df['points_gap'].mean():.1f} points")

# Champion frequency
print(f"\n🏆 Most Successful Teams (Championships):")
champion_counts = summary_df['champion'].value_counts()
for team, count in champion_counts.head(5).items():
    print(f"  • {team}: {count} titles")

🏆 Multi-Season Title Race Analysis
📊 Processing 2006/07: 132 matches, 12 teams
📊 Processing 2007/08: 132 matches, 12 teams
📊 Processing 2008/09: 132 matches, 13 teams
📊 Processing 2009/10: 239 matches, 16 teams
📊 Processing 2010/11: 234 matches, 16 teams
📊 Processing 2009/10: 239 matches, 16 teams
📊 Processing 2010/11: 234 matches, 16 teams
📊 Processing 2011/12: 240 matches, 16 teams
📊 Processing 2012/13: 182 matches, 14 teams
📊 Processing 2011/12: 240 matches, 16 teams
📊 Processing 2012/13: 182 matches, 14 teams
📊 Processing 2013/14: 182 matches, 14 teams
📊 Processing 2014/15: 181 matches, 14 teams
📊 Processing 2013/14: 182 matches, 14 teams
📊 Processing 2014/15: 181 matches, 14 teams
📊 Processing 2015/16: 182 matches, 14 teams
📊 Processing 2016/17: 182 matches, 14 teams
📊 Processing 2017/18: 181 matches, 14 teams
📊 Processing 2015/16: 182 matches, 14 teams
📊 Processing 2016/17: 182 matches, 14 teams
📊 Processing 2017/18: 181 matches, 14 teams
📊 Processing 2018/19: 182 matches, 14 tea

,season,champion,champion_points,runner_up,runner_up_points,points_gap,leadership_changes,total_rounds
0,2006/07,Beitar Jerusalem,43,Hapoel Tel Aviv,39,4,0,22
1,2007/08,Beitar Jerusalem,51,Maccabi Netanya,40,11,0,22
2,2008/09,Maccabi Haifa,42,Hapoel Tel Aviv,41,1,4,19
3,2009/10,Maccabi Haifa,77,Hapoel Tel Aviv,71,6,2,30
4,2010/11,Maccabi Haifa,70,Hapoel Tel Aviv,62,8,3,30
5,2011/12,Ironi Kiryat Shmona,66,Hapoel Tel Aviv,53,13,6,30
6,2012/13,Maccabi Tel Aviv,59,Maccabi Haifa,49,10,4,26
7,2013/14,Maccabi Tel Aviv,66,Hapoel Be'er Sheva,59,7,3,26
8,2014/15,Maccabi Tel Aviv,55,Hapoel Be'er Sheva,49,6,5,26
9,2015/16,Hapoel Be'er Sheva,64,Maccabi Tel Aviv,61,3,1,26



✅ Summary saved to: c:\Users\nitib\dev-lab\ligat_haal_project\ligat_haal_project\notebooks\data\interim\title_race_summary.csv

📈 Interesting Statistics:
  • Most competitive season: 2011/12 (6 leadership changes)
  • Most dominant win: 2018/19 (22 points gap)
  • Closest finish: 2008/09 (1 points gap)
  • Average leadership changes per season: 2.4
  • Average points gap: 6.0 points

🏆 Most Successful Teams (Championships):
  • Maccabi Haifa: 6 titles
  • Maccabi Tel Aviv: 6 titles
  • Hapoel Be'er Sheva: 5 titles
  • Beitar Jerusalem: 2 titles
  • Ironi Kiryat Shmona: 1 titles


## 7. Detailed Season Export

Export detailed round-by-round standings for each season to interim folder for further analysis.

In [19]:
# Export detailed standings for each season
print("💾 Exporting detailed round-by-round standings...")
print("="*80)

export_count = 0
for season_str in all_seasons:
    standings_by_round, leadership_changes = calculate_league_table_by_round(all_matches, season_str)
    
    if standings_by_round:
        # Combine all rounds into single DataFrame
        all_rounds = []
        for round_num, standings in standings_by_round.items():
            standings_copy = standings.copy()
            standings_copy['round'] = round_num
            standings_copy['season'] = season_str
            all_rounds.append(standings_copy)
        
        combined_standings = pd.concat(all_rounds, ignore_index=True)
        
        # Reorder columns for clarity
        cols = ['season', 'round', 'position', 'team', 'played', 'won', 'drawn', 'lost', 
                'gf', 'ga', 'gd', 'points']
        combined_standings = combined_standings[cols]
        
        # Save to file
        season_file = INTERIM_DIR / f"standings_by_round_{season_str.replace('/', '_')}.csv"
        combined_standings.to_csv(season_file, index=False, encoding='utf-8-sig')
        export_count += 1

print(f"\n✅ Exported {export_count} season files to: {INTERIM_DIR}")
print(f"   Files named: standings_by_round_YYYY_YY.csv")
print("\n💡 These files contain complete round-by-round standings for each season.")
print("   Use them for detailed analysis, visualizations, or further processing.")

💾 Exporting detailed round-by-round standings...
📊 Processing 2006/07: 132 matches, 12 teams
📊 Processing 2007/08: 132 matches, 12 teams
📊 Processing 2008/09: 132 matches, 13 teams
📊 Processing 2009/10: 239 matches, 16 teams
📊 Processing 2008/09: 132 matches, 13 teams
📊 Processing 2009/10: 239 matches, 16 teams
📊 Processing 2010/11: 234 matches, 16 teams
📊 Processing 2011/12: 240 matches, 16 teams
📊 Processing 2010/11: 234 matches, 16 teams
📊 Processing 2011/12: 240 matches, 16 teams
📊 Processing 2012/13: 182 matches, 14 teams
📊 Processing 2013/14: 182 matches, 14 teams
📊 Processing 2012/13: 182 matches, 14 teams
📊 Processing 2013/14: 182 matches, 14 teams
📊 Processing 2014/15: 181 matches, 14 teams
📊 Processing 2015/16: 182 matches, 14 teams
📊 Processing 2014/15: 181 matches, 14 teams
📊 Processing 2015/16: 182 matches, 14 teams
📊 Processing 2016/17: 182 matches, 14 teams
📊 Processing 2017/18: 181 matches, 14 teams
📊 Processing 2016/17: 182 matches, 14 teams
📊 Processing 2017/18: 181 m

---

## Summary

This notebook analyzes the **title race** in Ligat Ha'al across all available seasons.

### Key Outputs:

1. **Season-by-season leadership analysis** - Who led after each round
2. **Leadership change tracking** - How many times the leader changed
3. **Champion statistics** - Points totals, gaps to runner-up
4. **Historical comparison** - Most competitive vs. most dominant seasons

### Data Files Created:

- `interim/title_race_summary.csv` - Summary statistics for all seasons
- `interim/standings_by_round_YYYY_YY.csv` - Detailed round-by-round standings per season

### Important Notes:

- Round numbers are **estimated** since Wikipedia doesn't include match dates
- Analysis based on regular season matches from Wikipedia results matrix
- Team names are normalized for consistency across all seasons
- Standings calculated using standard rules: 3 pts win, 1 pt draw, 0 pts loss
- Tiebreakers: points → goal difference → goals scored

### Next Steps:

1. Visualize leadership changes over time
2. Identify patterns in title races (comebacks, dominant leads, etc.)
3. Correlate with attendance data
4. Analyze home/away performance of title contenders
5. Statistical modeling of championship probability by round